In [10]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import os

In [50]:
# --- 1. Definir rutas del proyecto (Usando TU método) ---
# Este es el bloque que has confirmado que funciona en tu entorno.
try:
    ruta_base = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
    ruta_gold = os.path.join(ruta_base, "GOLD")
    print(f"Ruta GOLD configurada exitosamente en: {ruta_gold}")
except Exception as e:
    print(f"Error configurando las rutas: {e}")

Ruta GOLD configurada exitosamente en: C:\Users\elezr\Datalquemy\Proyectos\CNH\GOLD


In [56]:
# --- 2. Cargar el dataset de los 10 campos ---
file_path_input = os.path.join(ruta_gold, "top_10_campos_timeseries.parquet")
df_top_10 = pd.read_parquet(file_path_input)
print(f"\nArchivo '{os.path.basename(file_path_input)}' cargado exitosamente.")


Archivo 'top_10_campos_timeseries.parquet' cargado exitosamente.


In [62]:
# --- 4. INGENIERÍA DE CARACTERÍSTICAS (VERSIÓN MEJORADA) ---
print("\nIniciando ingeniería de características avanzada...")
    
df_campo['produccion_lag_1'] = df_campo['petroleo_mbd'].shift(1)
df_campo['produccion_lag_6'] = df_campo['petroleo_mbd'].shift(6)
df_campo['produccion_lag_12'] = df_campo['petroleo_mbd'].shift(12)
df_campo['produccion_roll_mean_6'] = df_campo['petroleo_mbd'].rolling(window=6).mean()
df_campo['produccion_roll_std_6'] = df_campo['petroleo_mbd'].rolling(window=6).std()
df_campo['mes'] = df_campo.index.month
df_campo['trimestre'] = df_campo.index.quarter
df_campo['año'] = df_campo.index.year


Iniciando ingeniería de características avanzada...


In [64]:
# --- 5. Preparación Final y Guardado ---
df_modelo = df_campo.dropna()
print(f"\nDataFrame final con {df_modelo.shape[0]} filas listas para el modelo.")
    
nombre_archivo_salida = "maloob_features_for_training.parquet"
ruta_completa_salida = os.path.join(ruta_gold, nombre_archivo_salida)
df_modelo.to_parquet(ruta_completa_salida)

print(f"\n¡Proceso completado!")
print(f"DataFrame con características para Maloob guardado en:")
print(f"{ruta_completa_salida}")


DataFrame final con 97 filas listas para el modelo.

¡Proceso completado!
DataFrame con características para Maloob guardado en:
C:\Users\elezr\Datalquemy\Proyectos\CNH\GOLD\maloob_features_for_training.parquet
